# Sistemas Especialista

In [3]:
import experta

base_conhecimento = {
    "sol": {
        "quente": "Ir à Praia",
        "ameno": "Fazer uma caminhada no parque"
    },
    "nublado": {
        "quente": "Visitar um parente com ar condicionado",
        "ameno": "Ficar em casa e tomar café"
    },
    "chuva": {
        "quente": "Ficar na chuva",
        "ameno": "Ficar em casa comendo pipoca"
    }
}

## Motor de inferência (SIMPLES)

In [4]:
def motor_inferencia_clima(fatos):
  clima = fatos.get('clima')
  temperatura = fatos.get('temperatura')

  if clima in base_conhecimento and temperatura in base_conhecimento[clima]:
      return base_conhecimento[clima][temperatura]
  else:
      return "Não tenho recomendação para esta combinação de clima e temperatura."

### Simulação

In [5]:
fatos = {
  "clima": "chuva",
  "temperatura": "ameno"
}

conclusao = motor_inferencia_clima(fatos)
print(f"Fatos: {fatos}")
print(f"Recomendação do sistema: {conclusao}")

Fatos: {'clima': 'chuva', 'temperatura': 'ameno'}
Recomendação do sistema: Ficar em casa comendo pipoca


## Forward e Backward

In [6]:
fatos = ["tem_pelos", "voa", "produz_leite"]
regras = [
  {
    "se": ["tem_penas", "voa"],
    "entao": "e_pessaro"
  },
  {
    "se": ["tem_penas", "pode_cantar"],
    "entao": "e_canario"
  },
  {
    "se": ["tem_pelos", "produz_leite"],
    "entao": "e_mamifero"
  },
  {
    "se": ["tem_pelos", "voa"],
    "entao": "e_morcego"
  },
  {
    "se": ["tem_pelos", "pode_cantar"],
    "entao": "e_foca"
  },
  {
    "se": ["tem_pelos", "late"],
    "entao": "e_cachorro"
  }
]

## Motor de Inferência (Forward)

In [7]:
def motor_inferencia_forward(fatos_init, regras):
    fatos_derivados = list(fatos_init)
    novo_fato = True

    while novo_fato:
        novo_fato = False
        for regra in regras:
            condicao_satisfeita = all(condicao in fatos_derivados for condicao in regra["se"])

            if condicao_satisfeita and regra["entao"] not in fatos_derivados:
                fatos_derivados.append(regra["entao"])
                print(f"Regra disparada: SE {regra['se']} ENTAO {regra['entao']}")
                print(f"Fatos adicionado: {regra['entao']}")
                novo_fato = True

    return fatos_derivados

### Simulação

In [8]:
print(f"Fatos iniciais: {fatos}")
fatos_finais = motor_inferencia_forward(fatos, regras)
print(f"Fatos finais: {fatos_finais}")

Fatos iniciais: ['tem_pelos', 'voa', 'produz_leite']
Regra disparada: SE ['tem_pelos', 'produz_leite'] ENTAO e_mamifero
Fatos adicionado: e_mamifero
Regra disparada: SE ['tem_pelos', 'voa'] ENTAO e_morcego
Fatos adicionado: e_morcego
Fatos finais: ['tem_pelos', 'voa', 'produz_leite', 'e_mamifero', 'e_morcego']


# Biblioteca Experta

In [9]:
from experta import *
class Caracteristica(Fact):
  "Representa uma característica observada"
  pass

class Animal(KnowledgeEngine):
    @DefFacts()
    def fatos_iniciais(self):
        yield Caracteristica("tem_penas")
        yield Caracteristica("voa")
        yield Caracteristica("pode_cantar")
        print("Fatos iniciais carregados.")

    @Rule(Caracteristica("tem_penas"), Caracteristica("voa"))
    def regra_e_passaro(self):
        print("Regra disparada: SE tem_penas e voa ENTAO e_passaro")
        self.declare(Fact(animal="e_passaro"))

    @Rule(Fact(animal="e_passaro"), Caracteristica("pode_cantar"))
    def regra_e_canario(self):
        print("Regra disparada: SE e_passaro e pode_cantar ENTAO e_canario")
        self.declare(Fact(animal="canário"))

    @Rule(Fact(animal=MATCH.tipo))
    def print_resultado(self, tipo):
        if tipo == "canário":
            print(f"Conclusão final obtida, o tipo de animal é: {tipo}")

animal = Animal()
animal.reset()
animal.run()

Fatos iniciais carregados.
Regra disparada: SE tem_penas e voa ENTAO e_passaro
Regra disparada: SE e_passaro e pode_cantar ENTAO e_canario
Conclusão final obtida, o tipo de animal é: canário


## Verficação de Doença

In [10]:
class Sintoma(Fact):
  pass

class SistemaDiagnostico(KnowledgeEngine):
  @DefFacts()
  def sintomas_observados(self):
    # Estados: normal, leve, grave, muito_grave
    yield Sintoma(tipo="tosse", estado="grave")
    yield Sintoma(tipo="enxaqueca", estado="normal")
    yield Sintoma(tipo="fadiga", estado="muito_grave")
    yield Sintoma(tipo="diarreia", estado="normal")
    yield Sintoma(tipo="febre", estado="leve")
    yield Sintoma(tipo="dor_muscular", estado="normal")
    print("Fatos iniciais carregados.")
  
  @Rule(Sintoma(tipo="tosse", estado="grave"), Sintoma(tipo="febre", estado="leve"))
  def diagnostico_gripe(self):
    print("Regra disparada: SE tosse(grave) e febre(leve) ENTAO diagnóstico: gripe")
    self.declare(Fact(diagnostico="gripe"), solucao="descansar e tomar bebidas quentes")

  @Rule(Sintoma(tipo="fadiga", estado="muito_grave"), Sintoma(tipo="diarreia", estado="normal"))
  def diagnostico_anemia(self):
    print("Regra disparada: SE fadiga(muito_grave) e diarreia(normal) ENTAO diagnóstico: anemia")
    self.declare(Fact(diagnostico="anemia"), solucao="aumentar o consumo de ferro")

  @Rule(Fact(Sintoma(tipo="enxaqueca", estado="grave")), Sintoma(tipo="dor_muscular", estado="muito_grave"))
  def diagnostico_dengue(self):
    print("Regra disparada: SE enxaqueca(grave) e dor_muscular(grave) ENTAO diagnóstico: dengue")
    self.declare(Fact(diagnostico="dengue"), solucao="hidratar e repousar")
  
  @Rule(Fact(diagnostico=MATCH.tipo), solucao=MATCH.solucao)
  def resultados(self, tipo):
    print(f"Diagnóstico final obtido: {tipo}, recomendação médica: {self.solucao}")

TypeError: __new__() got an unexpected keyword argument 'solucao'

In [11]:
# Painel interativo
def interface_usuario():
  sistema = SistemaDiagnostico()

  print("Bem-vindo ao Sistema de Diagnóstico Médico")
  
  # pergunta 1
  print("Pergunta 1: Qual o estado da sua tosse? (normal, leve, grave, muito_grave)")
  tosse_estado = input().strip().lower()

  print("Pergunta 2: Qual o estado da sua febre? (normal, leve, grave, muito_grave)")
  febre_estado = input().strip().lower()

  print("Pergunta 3: Qual o estado da sua fadiga? (normal, leve, grave, muito_grave)")
  fadiga_estado = input().strip().lower()

  print("Pergunta 4: Qual o estado da sua diarreia? (normal, leve, grave, muito_grave)")
  diarreia_estado = input().strip().lower()

  print("Pergunta 5: Qual o estado da sua enxaqueca? (normal, leve, grave, muito_grave)")
  enxaqueca_estado = input().strip().lower()

  print("Pergunta 6: Qual o estado da sua dor muscular? (normal, leve, grave, muito_grave)")
  dor_muscular_estado = input().strip().lower()

  sistema.reset()
  sistema.declare(Sintoma(tipo="tosse", estado=tosse_estado))
  sistema.declare(Sintoma(tipo="febre", estado=febre_estado))
  sistema.declare(Sintoma(tipo="fadiga", estado=fadiga_estado))
  sistema.declare(Sintoma(tipo="diarreia", estado=diarreia_estado))
  sistema.declare(Sintoma(tipo="enxaqueca", estado=enxaqueca_estado))
  sistema.declare(Sintoma(tipo="dor_muscular", estado=dor_muscular_estado))

interface_usuario()

NameError: name 'SistemaDiagnostico' is not defined